In [16]:
#Load model
import torch
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)
#print(ssd_model --help)

Using cache found in C:\Users\komleva-ep/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [17]:
def create_model(num_classes=4):

    model = resnet50(pretrained=True)
    num_feats = model.fc.in_features
    model.fc = nn.Linear(num_feats, num_classes)
    return model


In [18]:
def get_dataloaders(dataset_dirs):

    train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    valid_transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    train_dataset = ImageFolderWithSubfolders(root='../data/input/train',
                                              transform=train_transform,
                                              subfolders=dataset_dirs)
    valid_dataset = ImageFolderWithSubfolders(root='../data/input/valid',
                                              transform=valid_transform,
                                              subfolders=dataset_dirs)

    train_dataloader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
    valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=64, shuffle=False)

    return train_dataloader, valid_dataloader


In [19]:
def eval_model(model, valid_dataloader, plot=False):
    print(valid_dataloader.dataset.class_to_idx)
    with torch.no_grad():
        model.eval()
        result = 0
        n = 0
        val_preds = np.zeros(len(valid_dataloader.dataset))
        val_labels = np.zeros(len(valid_dataloader.dataset))
        start = 0
        general_classifier = GeneralClassifier()
        for images, labels in valid_dataloader:
            batch_size = images.size(0)
            n += batch_size
            #images = images.cuda()
            #img = preprocess_for_ssd(general_classifier, images)
            #print(img)
            #pred = F.softmax(model(img))
            pred  = 0.5#person_predict(general_classifier, images)
            prediction = torch.argmax(pred, 1)

            val_preds[start:start + batch_size] = prediction.cpu().numpy()
            val_labels[start:start + batch_size] = labels.numpy()
            start += batch_size

    print('Precision: ', precision_score(val_labels, val_preds, average='weighted'))
    print('Recall: ', recall_score(val_labels, val_preds, average='weighted'))
    print('F1 score: ', f1_score(val_labels, val_preds, average='weighted'))

    if plot:
        cm = confusion_matrix(val_labels, val_preds,
                              labels=list(valid_dataloader.dataset.class_to_idx.values()))
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                      display_labels=list(valid_dataloader.dataset.class_to_idx.keys()))
        #disp.plot()
        
        disp.plot(cmap = plt.cm.Blues)
        plt.xticks(rotation=45)
        plt.show()

In [20]:
def train_model(model, train_dataloader, valid_dataloader, start_epoch=0, num_epochs=5, plot=False):
    #model.cuda()
    #optimizer = optim.Adam(params=model.fc.parameters(), lr=0.0001, betas=(0.9, 0.999))
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    criterion = nn.CrossEntropyLoss()
    losses = []
    for epoch in range(start_epoch,num_epochs):
        print(epoch)
        model.train()
        for images, labels in tqdm(train_dataloader):
            optimizer.zero_grad()
            #images = images.cuda()
            #labels = labels.cuda()
            pre_img = preprocess_for_ssd(images)
            pred = model(pre_img)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        eval_model(model, valid_dataloader)
        #torch.save(model, '../checkpoints/text_classifier_{}.pt'.format(epoch+1))
        torch.save(model, '../checkpoints/text_classifier_2cats_{}.pt'.format(epoch+1))
        if plot:
            plt.plot(losses)

    return model

In [21]:
def person_predict(self, x):
        with torch.no_grad():
            img = self.preprocess_for_ssd(x)
            ssd_predictions = self.person_detector(img)
            ssd_results = self.person_detection_utils.decode_results(ssd_predictions)
            bboxes, classes, confidences = self.person_detection_utils.pick_best(ssd_results[0], 0.40)
            max_person_conf = 0
            for cat, conf in zip(classes, confidences):
                if cat == 1 and conf > max_person_conf:
                    max_person_conf = conf
        return 'person: {:.2f}'.format(max_person_conf)

    
    
    
def preprocess_for_ssd( image):
        #print(image)
        #im = Image.open("a0.01dg0.30d0.08g0.02h0.54n0.03p0.02gr0.00ne1.00per0.62.png")
        #np_im = numpy.array(im)
        #print (np_im.shape)

        #np_im = np_im - 18
        #img = Image.fromarray(np_im)
        img = Image.fromarray(image)
        transform = transforms.Compose([
            transforms.Resize((300, 300)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        img = transform(img)
        img = img.to(self.device)
        img = img.unsqueeze(0)
        return img    

In [22]:
print((613-160)/(613-160+160))

0.7389885807504079


In [23]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,\
    accuracy_score, f1_score, precision_score, recall_score, classification_report
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from PIL import Image
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet50

from tqdm import tqdm

from datasets import ImageFolderWithSubfolders


if __name__=='__main__':
    dataset_dirs = ['pearson', 'no_pearson']
    train_dataloader, valid_dataloader = get_dataloaders(dataset_dirs)
    #model = create_model(7)#.cuda()
    model = ssd_model#torch.load('../checkpoints/best_nswf_classifeir.pt')#.cuda()
    #model = train_model(model, train_dataloader, valid_dataloader, start_epoch=0, num_epochs=15)
    #eval_model(model, valid_dataloader, plot=True)
    model = train_model(model, train_dataloader, valid_dataloader, start_epoch=0, num_epochs=10)
    
    #eval_model_2clasees(model, valid_dataloader, negative_classes=['drugs', 'gore', 'hentai', 'porn', 'drawing_gore',  'animals'],
    #                    positive_classes=['neutral'], plot=True)
    #torch.save(model.state_dict(), '../checkpoints/model_7dirs_5.pth')


  0%|          | 0/4 [00:00<?, ?it/s]

0


  0%|          | 0/4 [00:00<?, ?it/s]


AttributeError: 'Tensor' object has no attribute '__array_interface__'

In [ ]:
from PIL import Image
import numpy
im = Image.open("a0.01dg0.30d0.08g0.02h0.54n0.03p0.02gr0.00ne1.00per0.62.png")
np_im = numpy.array(im)
#print (np_im.shape)

#np_im = np_im - 18
new_im = Image.fromarray(np_im)
#print(np_im)
new_im.save("numpy_altered_sample2.png")

In [ ]:
#######################
####################
import cv2
#####################
from general_model import GeneralClassifier 
general_classifier = GeneralClassifier()
full_filename = r'a0.01dg0.30d0.08g0.02h0.54n0.03p0.02gr0.00ne1.00per0.62.png'
x = cv2.imread(full_filename)
cv2. imshow("IMG",x)
person_string = person_predict(general_classifier,x)
print(person_string)
print()

In [ ]:
pip install imutils

In [ ]:
s = ''
for i in range(111):
    s= s+'0'+ ','
print(s)    

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

y_actu = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0,0,1,1, 1, 1,0,0,1]
y_pred = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1, 1, 0, 0 , 0 ,0,1,1 ,0 ]
labels = np.unique(y_pred)
cm = confusion_matrix(y_actu, y_pred, labels =labels)
#cm = confusion_matrix(val_labels, val_preds, labels=list(valid_dataloader.dataset.class_to_idx.values()))
#cm = confusionchart([0 ,2 ],{'apple','watermelon'})
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['not people', 'people'])
#disp.plot()

#fig = plt.figure()        
disp.plot(cmap = plt.cm.Blues)
plt.title('SSD confusion matrix')
#plt.xticks(rotation=45)
#fig.suptitle('test title', fontsize=20)
plt.show()
plt.savefig('cm.png')


In [ ]:
# Imports
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_classes = 10 
learning_rate = 1e-3
batch_size = 1024
num_epochs = 5

# Simple Identity class that let's input pass without changes
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
# Load pretrain model & modify it
model = #torchvision.models.vgg16(pretrained=True)

# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.
for param in model.parameters():
    param.requires_grad = False
    
model.avgpool = Identity()
model.classifier = nn.Sequential(nn.Linear(512, 100),
                                 nn.ReLU(),
                                 nn.Linear(100, num_classes))
model.to(device)


# Load Data
train_dataset = datasets.CIFAR10(root='dataset/', train=True, 
                                 transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Loss and optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    losses = []
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        losses.append(loss.item())
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
    
    print(f'Cost at epoch {epoch} is {sum(losses)/len(losses):.5f}')

# Check accuracy on training & test to see how good our model

def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
    model.train()

check_accuracy(train_loader, model)


In [ ]:
conda install -c conda-forge ipywidgets
jupyter nbextension enable --py widgetsnbextension